# O PROBLEMA

Imagine agora que você foi contratado(a) como Expert em Data Analytics por um grande hospital para entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença.

Apesar de ser contratado(a) agora, a sua área observou que a utilização do estudo do PNAD-COVID 19 do IBGE seria uma ótima base para termos boas respostas ao problema proposto, pois são dados confiáveis. Porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas.

É sempre bom ressaltar que há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

## PNAD-COVID-19 do IBGE

O Head de Dados pediu para que você entrasse na base de dados do PNAD-COVID-19 do IBGE e organizasse esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

- a. Utilização de no máximo 20 questionamentos realizados na pesquisa;
- b. Utilizar 3 meses para construção da solução;
- c. Caracterização dos sintomas clínicos da população;
- d. Comportamento da população na época da COVID-19;
- e. Características econômicas da Sociedade;

Seu objetivo será trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19.

- Dica: Leiam com atenção a base de dados e toda a documentação que o site o PNAD – Covid19 traz, principalmente os dicionários, que ajudam e muito no entendimento da Base de Dados.
- Dica 2: Utilizem o que já foi ensinado e consolidado nas outras fases para apresentar a resolução do projeto.

Lembre-se de que você poderá apresentar o desenvolvimento do seu projeto durante as lives com docentes. Essa é uma boa oportunidade para discutir sobre as dificuldades encontradas e pegar dicas valiosas com especialistas e colegas de turma.

Link para a base: https://www.ibge.gov.br/estatisticas/investigacoes-experimentais/estatisticas-experimentais/27946-divulgacao-semanal-pnadcovid1?t=downloads&utm_source=covid19&utm_medium=hotsite&utm_campaign=covid_19

Outras fontes:

https://www.gov.br/saude/pt-br/assuntos/noticias/2021/janeiro/entenda-a-ordem-de-vacinacao-contra-a-covid-19-entre-os-grupos-prioritarios

https://www.gov.br/saude/pt-br/assuntos/coronavirus/vacinas/esquemas-vacinais

### Instalando dependências e bibliotecas

In [ ]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #instalando java 8
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz #baixando Spark
!tar xf spark-3.1.2-bin-hadoop2.7.tgz #descompactando Spark
!pip install -q findspark

In [ ]:
#criando arquivo requirements.txt
'''%%writefile requirements.txt

asttokens==2.4.0
attrs==23.1.0
backcall==0.2.0
certifi==2023.7.22
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
comm==0.1.4
debugpy==1.8.0
decorator==5.1.1
executing==2.0.0
fastjsonschema==2.18.1
findspark==2.0.1
fiona==1.9.5
geopandas==0.13.2
greenlet==3.0.0
importlib-metadata==6.8.0
importlib-resources==6.1.0
ipykernel==6.25.2
ipython==8.12.3
jedi==0.19.1
jsonschema==4.19.1
jsonschema-specifications==2023.7.1
jupyter-client==8.4.0
jupyter-core==5.4.0
matplotlib-inline==0.1.6
mysql-connector-python==8.1.0
nbformat==5.9.2
nest-asyncio==1.5.8
numpy==1.24.4
packaging==23.2
pandas==2.0.3
pandasql==0.7.3
parso==0.8.3
pexpect==4.8.0
pickleshare==0.7.5
pkgutil-resolve-name==1.3.10
platformdirs==3.11.0
plotly==5.17.0
polars==0.19.11
prompt-toolkit==3.0.39
protobuf==4.21.12
psutil==5.9.5
ptyprocess==0.7.0
pure-eval==0.2.2
py4j==0.10.9.7
Pygments==2.16.1
PyMySQL==1.1.0
pyproj==3.5.0
pyspark==3.5.0
python-dateutil==2.8.2
pytz==2023.3.post1
pyzmq==25.1.1
referencing==0.30.2
rpds-py==0.10.6
shapely==2.0.2
six==1.16.0
SQLAlchemy==2.0.22
stack-data==0.6.3
tenacity==8.2.3
tornado==6.3.3
traitlets==5.11.2
typing-extensions==4.8.0
tzdata==2023.3
wcwidth==0.2.8
zipp==3.17.0'''

In [ ]:
'''!pip install -r requirements.txt'''

### Importando bibliotecas

In [ ]:
import warnings
warnings.filterwarnings(action = 'ignore')

In [ ]:
import os


os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.1.2-bin-hadoop2.7'

In [ ]:
import findspark

findspark.init()

In [ ]:
#bibliotecas para conexão com o banco de dados
from sqlalchemy import create_engine
from sqlalchemy.sql import text

#bibliotecas dataviz
import pandas as pd
import polars as pl
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
findspark.init()

### Criando sessão Spark

In [15]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.master('local[*]').appName('Iniciando com Spark').getOrCreate()

In [16]:
spark

### Importando dados

In [17]:
from google.colab import drive


drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
import zipfile


zipfile.ZipFile('/content/drive/MyDrive/cursos/postech/postech_techchallenge_fase_3/dados/2023-10-27_pnad_covid_view.zip', 'r').extractall('/content/drive/MyDrive/cursos/postech/postech_techchallenge_fase_3/dados')

In [55]:
path = '/content/drive/MyDrive/cursos/postech/postech_techchallenge_fase_3/dados/2023-10-27_pnad_covid_view.csv'
df = spark.read.csv(path, sep=',', inferSchema=True, header=True)

In [71]:
df.show(5)

2650459 530091


In [ ]:
df.printSchema()

### Dicionários

In [ ]:
estados = {
    'Acre': 'AC',
    'Alagoas': 'AL',
    'Amapá': 'AP',
    'Amazonas': 'AM',
    'Bahia': 'BA',
    'Ceará': 'CE', 
    'Distrito Federal': 'DF',
    'Espírito Santo': 'ES',
    'Goiás': 'GO',
    'Maranhão': 'MA',
    'Mato Grosso': 'MT',
    'Mato Grosso do Sul': 'MS',
    'Minas Gerais': 'MG',
    'Pará': 'PA',
    'Paraíba': 'PB',
    'Paraná': 'PR',
    'Pernambuco': 'PE',
    'Piauí': 'PI', 
    'Rio de Janeiro': 'RJ',
    'Rio Grande do Norte': 'RN',
    'Rio Grande do Sul': 'RS',
    'Rondônia': 'RO',
    'Roraima': 'RR',
    'Santa Catarina': 'SC',
    'São Paulo': 'SP',
    'Sergipe': 'SE',
    'Tocantins': 'TO'
}

regioes = {
    'AC': 'Norte',
    'AL': 'Nordeste',
    'AP': 'Norte',
    'AM': 'Norte',
    'BA': 'Nordeste',
    'CE': 'Nordeste',
    'DF': 'Centro-Oeste',
    'ES': 'Sudeste',
    'GO': 'Centro-Oeste',
    'MA': 'Nordeste',
    'MT': 'Centro-Oeste',
    'MS': 'Centro-Oeste', 
    'MG': 'Sudeste',
    'PA': 'Norte',
    'PB': 'Nordeste',
    'PR': 'Sul',
    'PE': 'Nordeste',
    'PI': 'Nordeste',
    'RJ': 'Sudeste',
    'RN': 'Nordeste',
    'RS': 'Sul',
    'RO': 'Norte',
    'RR': 'Norte',
    'SC': 'Sul',
    'SP': 'Sudeste',
    'SE': 'Nordeste',
    'TO': 'Norte'
}

### Adicionando colunas

In [ ]:
df = df.withColumn('sigla', df.uf).replace(to_replace=estados, subset=['sigla']) #adicionando sigla dos estados
df = df.withColumn('regiao', df.sigla).replace(to_replace=regioes, subset=['regiao']) #adicionando regioes dos estados
df.show(5, truncate=False)

### Análise exploratória